**Initialization**
- *obs. gitpython and pyvis packages installed in SA anaconda environment*
- Cloning Zeeguu-API

In [2]:
# For new environement, install the following packages
# install GitPython, and installs pyvis using the system's Python executable.
#gitpyhon is used to interact with git repositories, and pyvis is used to visualize the graph.
import sys
sys.version
!{sys.executable} -m pip install gitpython
!{sys.executable} -m pip install pyvis
!{sys.executable} -m pip install ast

  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\carl9\AppData\Local\Temp\pip-install-fovpukee\ast_0edaa66156b240beaef7b00b01a02f8d\setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen codecs>", line 918, in open
      FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\carl9\\AppData\\Local\\Temp\\pip-install-fovpukee\\ast_0edaa66156b240beaef7b00b01a02f8d\\AST/README'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generati

In [3]:
from git import Repo
import os
# GitPython is a library that allows us to work easily with git from Python
# https://gitpython.readthedocs.io/en/stable/tutorial.html

# Let's declare a var for the path where we're going to download a repository
# Warning: this must end in /
CODE_ROOT_FOLDER="C:/Users/carl9/OneDrive/Skrivebord/SA_Individual_ZeeguuAPI/reconstruction/zeeguu-api/"

# If the file exists, it means we've already downloaded
if not os.path.exists(CODE_ROOT_FOLDER):
  Repo.clone_from("https://github.com/zeeguu/api", CODE_ROOT_FOLDER)

**Helper Functions**

- **module_name_from_file_path()**  - *Extracts module from file path (.py files) e.g. ../core/model/user.py -> zeeguu.core.model.user*

In [4]:
# extracting a module name from a file name
def module_name_from_file_path(full_path):
    file_name = full_path[len(CODE_ROOT_FOLDER):]
    file_name = file_name.replace("/__init__.py","")
    file_name = file_name.replace("/",".")
    file_name = file_name.replace(".py","")
    return file_name

**Data Gathering**

Utilize a defined grammar to parse and extract dependencies efficiently. Employ Abstract Syntax Trees (AST) to overcome the limitations posed by regex-based methods. This includes:
1. Parse a file and generate AST (read file, then use ast package to generate AST from content)
2. Define the grammar rules for the dependency extraction
3. Traverse AST and collect dependencies based on grammar
4. *define abstractions on different kinds of AST nodes*
5. store dependencies, make em' ready for visualization!

In [6]:
import ast

# Parse a Python file and return an AST node. e.g. parse_file('example.py')
def file_to_ast_parser(f_path):
    with open(f_path, 'r') as file:
        code = file.read()
    return ast.parse(code, filename=f_path)

# Define Grammar rules for AST traversal of the pythcon code of a single file
# could also do some counting here for more advanced analysis  / metric aggregati
def collect_dependencies(abstract_syntax_tree):
    dependencies = set()
    module_attributes = dict()

    #Define node types/grammar rules for the AST traversal:
    def visit_Import(node):
        for alias in node.names:
            #collect the module names from the import statements (also handles the case of "import os" and "import os as my_os alias" woop)
            dependencies.add(alias.asname if alias.asname else alias.name)
            #print("ast Import: alias.name: ", alias.name)

    def visit_ImportFrom(node):
        if node.module:
            dependencies.add(node.module)
        # Collects the methods as well as all other attributes from the module
        for alias in node.names:
            module_attributes[alias.name] = node.module


    # Recursively traverse the AST from the root node to all its children
    def traverse(node):
        if isinstance(node, ast.Import):
            visit_Import(node)
        elif isinstance(node, ast.ImportFrom):
            visit_ImportFrom(node)
        for child in ast.iter_child_nodes(node):
            traverse(child)
        
    traverse(abstract_syntax_tree)
    print("all module attributes: ", module_attributes)
    used_dependencies = filter_dependencies(abstract_syntax_tree, dependencies, module_attributes)
    return dependencies, used_dependencies
    

def filter_dependencies(abstract_syntax_tree, dependencies, module_attributes):
    used_dependencies = set()

    def filter_traverse(node, dependencies):
        #ast.Name include variable names, function names, class names, module names, and other names used within the code.
        """ if isinstance(node, ast.Name):
            #node.id is the name of the Name node, e.g. "os" from "import os"
            if node.id in dependencies:
                print("ast Name: node.id: ", node.id)
                used_dependencies.add(node.id)
        
        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name) and node.func.id in dependencies:
                print("ast Call: node.func.id: ", node.func.id)
                used_dependencies.add(node.func.id) """
        
        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name): #and node.func.id in dependencies:
                if node.func.id in module_attributes:
                    print("ast Call: node.func.id: ", node.func.id)
                    used_dependencies.add(module_attributes[node.func.id])
                    
            if isinstance(node.func, ast.Attribute) and isinstance(node.func.value, ast.Name) and node.func.value.id in dependencies:
                print("ast Call: node.func.value.id: ", node.func.value.id)
                used_dependencies.add(node.func.value.id)

        for child in ast.iter_child_nodes(node):
            filter_traverse(child, dependencies)


    filter_traverse(abstract_syntax_tree, dependencies)
    return used_dependencies

#OOOOBS CARL. This does not solve the regex import problem. We are still limited by possible unused imports..
# hmm i think i will need a more sophisticated approach to this. From this i could also collect some metrics on the codebase hm


I dont think the above will work, i can't seem to manage the limitations using the ast module. It is much easier to make my own tokens...
I can check if the nodes of the AST are imports etc.. but then this is just the same as gathering information with reggex.. im still bottlenecked at 
filtering unused and uncommented dependencies. 
To do it well enough, i would have to go through all code and look at which imported methods are called (import from) and which imported modules are called. This is difficult with just ast as i fx. dont catch functions calls in other calls (eg. print(func()))
Also my current approach would only catch modules e.g. X.method(), not "from .. import method -> method()" as the method itself is not recorded as a dependency but only the module from which it is imported hmm.


Okay, now i also collect the functions that has been imported from modules


In [10]:
#Testing the traversal of the AST

ast_tree = file_to_ast_parser("C:/Users/carl9/OneDrive/Skrivebord/SA_Individual_ZeeguuAPI/test.py")
dependencies, used_dependencies = collect_dependencies(ast_tree)
#dependencies = collect_dependencies(ast_tree)
print("all dep", dependencies)
print("used dep", used_dependencies)
print("unused dep: ", dependencies - used_dependencies)

#Okay for tomorrow: with this approach i still dont filter outcommented imports and outcommented calls... 
# I think i will need to do some regex magic to filter out the outcommented lines before i parse the file, but then 
# i wont be able to trace the "unused" dependencies.. Maybe i need to tokenize everything on my own, this ast approach might get a little too complex

all module attributes:  {'request': 'urllib', 'load_configuration_or_abort': 'zeeguu.config.loader'}
ast Call: node.func.value.id:  np
ast Call: node.func.value.id:  ast
ast Call: node.func.id:  load_configuration_or_abort
all dep {'ast', 'zeeguu.config.loader', 'os', 'np', 'urllib'}
used dep {'ast', 'np', 'zeeguu.config.loader'}
unused dep:  {'urllib', 'os'}


**Visualization**
- polymetrics views?
